In [3]:
import os 

In [4]:
%pwd

'/home/ahmed/project/Kidney-Disease-Classification-Deep-learning-project/recsearch'

In [5]:
os.chdir('../')

In [6]:
%pwd

'/home/ahmed/project/Kidney-Disease-Classification-Deep-learning-project'

In [7]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class PrepareBasemodelConfig:
    root_dir: Path
    base_model: Path
    update_base_model: Path 
    param_image_size: list
    param_batch_size: int
    param_epochs: int 
    param_learning_rate: float
    param_classics: int
    param_weight:str
    param_include_top: bool



In [8]:
from project.constants import *
from project.utils import create_directories,read_yaml

In [9]:
class ConfigerationManager:
    def __init__(self, config=CONFIG_YAML_FILE, param=PARAM_YAML_FILE):
        self.config = read_yaml(config)
        self.param = read_yaml(param)
       
        create_directories(self.config.artifacts_root)

    def get_prepare_base_model(self)-> PrepareBasemodelConfig:
        config = self.config.prepare_base_model
        
        create_directories(config.root_dir)
        prepare_base_model_config = PrepareBasemodelConfig(
            root_dir=config.root_dir,
            base_model=config.base_model,
            update_base_model=config.update_base_model,
            param_image_size=self.param.IMAGE_SIZE, 
            param_batch_size=self.param.BATCH_SIZE,
            param_epochs=self.param.EPOCHS, 
            param_learning_rate=self.param.LEARNING_RATE, 
            param_classics=self.param.CLASSICS, 
            param_weight=self.param.WEIGHTS, 
            param_include_top=self.param.INCLUDETOP
            )
        
        return prepare_base_model_config

In [10]:

import tensorflow as tf

2025-11-23 19:58:47.457121: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-23 19:58:48.094117: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-23 19:58:52.234410: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [13]:
class PrepareBaseModel:
    def __init__(self,config=PrepareBasemodelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.applications.VGG16(
        include_top=self.config.param_include_top,
        weights=self.config.param_weight,
        input_shape=self.config.param_image_size,
        classes=self.config.param_classics,
        classifier_activation='softmax'
        )

        self.save_model(self.config.base_model,model=self.model)

    def update_base_model(self):
        self.full_model = self.prepare_model_layers(
            model=self.model,
            num_classes=self.config.param_classics,
            freeze_all=True, # base model freeze
            freeze_till=None, # classifier unfreeze
            learning_rate = self.config.param_learning_rate
        )
        self.save_model(self.config.update_base_model,model=self.full_model)



    @staticmethod
    def save_model(file_path:Path,model:tf.keras.Model):
        model.save(file_path)

    @staticmethod
    def prepare_model_layers(model,num_classes,learning_rate, freeze_all=False, freeze_till=None):
        """
        Freeze layers of a TensorFlow/Keras model based on configuration.

        Args:
            model (tf.keras.Model): The model whose layers need to be frozen.
            freeze_all (bool): If True, all layers will be frozen (trainable=False).
            freeze_till (int or None): 
                Number of layers from the end that should remain trainable.
                Example: freeze_till=10 → freeze all layers except the last 10.

        Returns:
            tf.keras.Model: The model with updated layer trainable settings.
        """

        # Freeze all layers
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False

        # Freeze layers up to a certain point
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(num_classes, activation='softmax')(flatten_in)

        # Combine base model and new classifier
        full_model = tf.keras.Model(inputs=model.input, outputs=prediction)
        full_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"]
        )

        full_model.summary()

        return full_model



In [14]:
try:
    config_manager = ConfigerationManager()
    prepare_base_model_config = config_manager.get_prepare_base_model()
    base_model = PrepareBaseModel(prepare_base_model_config)
    base_model.get_base_model()
    base_model.update_base_model()
except Exception as e:
    raise e

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)